## Especificação:

Nesta atividade você vai construir uma aplicação escolhendo uma das técnicas vistas no último módulo da disciplina. A atividade é individual. As possíveis aplicações são as seguintes:

    Classificação de notícias vs sátiras.
    Agrupamento de notícias.
    Análise de Sentimentos no Twitter.
    Sistemas de recomendação (filmes ou notícias).
    Detecção de viés em notícias.

As explicações do que fazer exatamente em cada uma foram dadas na aula do dia 16/07. 

O que entregar?

    Código documentado, de preferência link no github do notebook jupyter;
    Exemplos de funcionamento.

Critérios de correção

    Como de praxe, corretude e legibilidade.

Dicas

    Não falte as aulas de acompanhamento.
    Não deixe para última hora.
    Se faltou a aula do dia 16/07, procure com urgência o professor, monitores ou colegas trabalhando no tema de sua escolha para explicar o que deve ser feito.

Entregas com atraso

    Será descontado 10 pontos por dia de atraso


## Técnica escolhida: Agrupamento de notícias.

Breve descrição:
    
    No caso de notícias:

    Você vai usar os dados de notícias já utilizado em outras atividades;
    O algoritmo é basicamente o KNN;
    Passe uma notícia como consulta e retorne as top-5 mais similares (em termos de coseno ou produto escalar);
    A entrega é a mesma que filmes com exceção do RMSE.



## Código - Início

#### Organização dos imports

In [115]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import nltk
import re
from os import path
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
from unicodedata import normalize
from nltk.tokenize import word_tokenize
import numpy as np
import random
from collections import Counter

#### Recuperando documento e seu conteudo

In [101]:
news = pd.read_csv("./estadao_noticias_eleicao.csv", encoding="utf-8")
content = news.titulo + " " + news.subTitulo + " " + news.conteudo
content = content.fillna("")

#### Passo 1: Transformando em vetor

In [45]:
count_vect = TfidfVectorizer()
X_train_counts = count_vect.fit_transform(content[:1000])

In [42]:
X_train_counts.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

#### Verificando os centroides

(1, 568.9466880895357)
(2, 518.573809049568)
(3, 510.56389839678286)
(4, 504.09518963145507)
(5, 513.0102224540027)
(6, 506.31849974937103)
(7, 503.4217057978357)
(8, 503.04567703386584)
(9, 493.8840564420236)
(10, 500.9842819932499)


In [12]:
wcss = []

for i in range(1, 11):
    kmeans = KMeans(n_clusters = i, init = 'random')
    kmeans.fit(X_train_counts.toarray())
    print (i,kmeans.inertia_)
    wcss.append(kmeans.inertia_)  
plt.plot(range(1, 11), wcss)
plt.title('O Metodo Elbow')
plt.xlabel('Numero de Clusters')
plt.ylabel('WSS') #within cluster sum of squares
plt.show()

#### Iniciando o k-means e agrupar os dados a partir do metodo Elbow

In [46]:
#Kmeans = 4
kmeans = KMeans(n_clusters = 4, init = 'random', max_iter=1000, n_init=10)
kmeans.fit(X_train_counts.toarray())

KMeans(algorithm='auto', copy_x=True, init='random', max_iter=1000,
    n_clusters=4, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

#### Elaborando os graficos - Centroide

In [14]:
plt.scatter(X_train_counts.toarray()[:, 0], X_train_counts.toarray()[:,1], s = 100, c = kmeans.labels_)
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 300, c = 'red',label = 'Centroids')
plt.title('Iris Clusters and Centroids')
plt.xlabel('SepalLength')
plt.ylabel('SepalWidth')
plt.legend()
 
plt.show()

#### Agrupando os novos dados

In [15]:
#python -m pip install wordcloud
#
#wordcloud = WordCloud(font_path='/Library/Fonts/Verdana.ttf',
#                          relative_scaling = 1.0,
#                          stopwords = {'to', 'of'} # set or space-separated string
#                          ).generate(content)
#plt.imshow(wordcloud)
#plt.axis("off")
#plt.show()

In [93]:
groups_dic = {}
groups_term_freq = {}

labels = kmeans.labels_

for i in range(1000):
    groups_dic.setdefault(labels[i], []).append(content[i])
    
for group in sorted(groups_dic.keys()):
    groups_dic[group] = ' '.join(groups_dic[group])
    words = [word.lower() for word in n.word_tokenize(groups_dic[group])]
    freq_words = Counter(words)
    groups_term_freq[group] = freq_words.most_common()
    wordcloud = WordCloud(background_color='white', max_font_size=40).generate(groups_dic[group])
    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.title('Word Cloud do grupo ' + str(group))
    plt.axis("off")

TypeError: expected string or buffer

In [104]:
def build(data):
    data["noticia"] = data.titulo + " " + data.subTitulo + " " + data.conteudo
    docs = []
    stop_words = set(nltk.corpus.stopwords.words('portuguese'))
    stop_words.add('e')
    stop_words.add('é')
    stop_words.add('ser')
    
    for index, doc in data.iterrows():
        word_tokens = word_tokenize(doc.noticia)
        filtered_sentence = [w for w in word_tokens if not w in stop_words]
        news = ' '.join(filtered_sentence)
        docs.append(clear_text(news))

    return docs

In [92]:
def clear_text(text):
    pattern = re.compile('[^a-zA-Z0-9]')
    text = normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
    return pattern.sub(' ', text)

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

In [103]:
## Simple WordCloud
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from wordcloud import WordCloud, STOPWORDS 

text = 'all your base are belong to us all of your base base base'

def generate_wordcloud(text): # optionally add: stopwords=STOPWORDS and change the arg below
    wordcloud = WordCloud(relative_scaling = 1.0,
                          stopwords = {'to', 'of'} # set or space-separated string
                          ).generate(text)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

generate_wordcloud(documents)

NameError: name 'documents' is not defined

In [114]:
file_csv = "https://raw.githubusercontent.com/wendleypf/recuperacao_informacao/master/projeto-ri/data-set/estadao_noticias_eleicao.csv"
data = pd.read_csv(file_csv, encoding = "utf-8")
data = data.replace(np.NAN, "")

num_lines = len(data)
# ~10%
size = int(num_lines / 10)
skip_idx = random.sample(range(1, num_lines), num_lines - size)
data = pd.read_csv(file_csv, skiprows=skip_idx, encoding = "utf-8")
data = data.replace(np.NAN, "")
documents = build(data)

TypeError: normalize() argument 2 must be unicode, not str